# Cu Cluster Reaction Network Analysis

This notebook performs a reaction network analysis for different configurations of a 4-atom copper cluster. We will explore various arrangements of copper atoms, calculate the transition states between them, and generate a reaction network graph.

## Overview of Steps
1. Import necessary libraries
2. Define function to create Cu clusters
3. Create initial structures
4. Perturb structures and save
5. Set up calculation parameters
6. Perform transition state search
7. Generate reaction network

Let's get started!

## 1. Import Necessary Libraries

First, we import the required Python libraries. These include:
- `numpy`: for numerical computations
- `ase`: for atomic simulation environment
- `net_finder`: for transition state search and reaction network analysis

In [ ]:
import numpy as np
from ase import Atoms
from ase.io import write
from ase.calculators.emt import EMT
from net_finder.paths import TransitionStateSearch
from net_finder.net import ReactionNetwork

## 2. Define Function to Create Cu Clusters

This function will create a cluster containing 4 copper atoms. It sets non-periodic boundary conditions and adds vacuum around the cluster to prevent interactions between periodic images.

In [ ]:
def create_cu_cluster(positions):
    cluster = Atoms('Cu4', positions=positions)
    cluster.set_pbc(False)  # Explicitly set to non-periodic
    cluster.center(vacuum=5.0)  # Add vacuum around and center the cluster
    return cluster

## 3. Create Initial Structures

We will create 4 different copper cluster configurations:
1. Tetrahedral configuration
2. Planar rhombus configuration
3. Linear configuration
4. Planar square configuration

These different configurations will serve as the starting points for our reaction network analysis.

In [ ]:
# Create initial structures (different arrangements of 4 copper atoms)
structures = [
    create_cu_cluster([
        (0, 0, 0),
        (2.5, 0, 0),
        (1.25, 2.165, 0),
        (1.25, 0.722, 2.165)
    ]),  # Tetrahedral configuration
    create_cu_cluster([
        (0, 0, 0),
        (2.5, 0, 0),
        (1.25, 2.165, 0),
        (3.75, 2.165, 0)
    ]),  # Planar rhombus configuration
    create_cu_cluster([
        (0, 0, 0),
        (2.5, 0, 0),
        (5.0, 0, 0),
        (7.5, 0, 0)
    ]),  # Linear configuration
    create_cu_cluster([
        (0, 0, 0),
        (2.5, 0, 0),
        (0, 2.5, 0),
        (2.5, 2.5, 0)
    ])  # Planar square configuration
]

## 4. Perturb Structures and Save

To increase diversity and simulate thermal motion in a real environment, we apply small random perturbations to each structure. Then, we save these structures in XYZ file format for subsequent analysis or visualization.

In [ ]:
# Slightly perturb each structure to increase diversity
for structure in structures:
    structure.positions += np.random.rand(*structure.positions.shape) * 0.1
    structure.center(vacuum=5.0)  # Re-center and ensure sufficient vacuum

# Save initial structures
for i, structure in enumerate(structures):
    write(f'initial/cu_cluster_{i}.xyz', structure)

print(f"Created and saved {len(structures)} initial structures, each with {len(structures[0])} copper atoms.")

## 5. Set Up Calculation Parameters

In this step, we set up three important sets of parameters:
1. EMT calculator: for calculating interatomic interactions
2. Dimer parameters: for transition state search
3. Run parameters: specifying how to execute the calculations

In [ ]:
# Set up EMT calculator
calc = {
    "calculator_imports": "from ase.calculators.emt import EMT",
    "calculator_setup": "calc = EMT()"
}

# Set up Dimer parameters
dimer_params = {
    'displacement_method': 'gauss',
    'displacement_radius': 0.5,
    'dimer_separation': 0.01
}

# Set up run parameters
run_params = {
    'run_mode': 'shell'
}

## 6. Perform Transition State Search

Now we use the TransitionStateSearch class to find transition states between different Cu cluster configurations. This process may take some time, depending on your computer's performance and the parameters set.

In [ ]:
# Create TransitionStateSearch object and run
ts_search = TransitionStateSearch(structures, calc, dimer_params, output_dir='output', **run_params)
ts_search.run(fmax=0.05, steps=100)

# Collect reaction paths
reaction_paths = ts_search.collect_reaction_paths()
print(f"Collected {len(reaction_paths)} reaction paths")

## 7. Generate Reaction Network

Finally, we use the reaction paths found to generate a reaction network. We set an energy threshold to filter out paths with too high energy. The generated network graph will be saved in the specified output directory.

In [ ]:
# Create ReactionNetwork object and generate graph
energy_cutoff = 10.0  # Set an appropriate energy threshold (in eV)
network = ReactionNetwork(reaction_paths, energy_cutoff)
network.generate_graph(output_dir='reaction_network_output')
print("Reaction network analysis complete, results saved in 'reaction_network_output' directory")

## Conclusion

Through this notebook, we have completed the following steps:
1. Created different configurations of Cu4 clusters
2. Perturbed and saved these structures
3. Set up necessary calculation parameters
4. Performed transition state search
5. Generated a reaction network

The generated reaction network graph can help us understand the transformation paths and energy barriers between different Cu4 cluster configurations. This is particularly useful for studying the structural evolution and catalytic performance of nanoparticles.

Next steps could include examining the output files in the 'reaction_network_output' directory, analyzing the generated reaction network graph, and adjusting parameters as needed for more in-depth studies.